In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_absolute_error as MAE, mean_squared_error as MSE, confusion_matrix
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from xgboost import XGBClassifier, XGBRegressor
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import scale, StandardScaler

In [3]:
all_new=pd.read_csv("/Users/Wei/Google Drive/Bentley/S3 MA710 Data mining/Data Mining group work/Final project/Google analytics data/all_new.csv")

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
all_new.head()

,Unnamed: 0,fullVisitorId,hits,pageviews,page_hits,visitor_number,diff_days,totaltransaction,log_revenue,transac_1_0,...,Internet.Explorer,Maxthon,Nokia.1,Opera,browernew.Others,Puffin,Safari,Seznam,YaBrowser,YE
0,0,4823595352351,1,1.0,1.000000,1,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,1,5103959234087,10,8.0,0.800000,1,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,2,10278554503158,11,8.0,0.727273,1,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,3,20424342248747,17,13.0,0.764706,1,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,4,26722803385797,3,2.0,0.666667,1,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0


In [5]:
all_new=all_new.iloc[:,1:]

In [6]:
all_new=all_new.set_index("fullVisitorId")

In [7]:
al=all_new.drop(["totaltransaction", "log_revenue"], axis=1)

In [8]:
al.head()

,hits,pageviews,page_hits,visitor_number,diff_days,transac_1_0,channel.other,channel.affiliates,channel.direct,channel.display,...,Internet.Explorer,Maxthon,Nokia.1,Opera,browernew.Others,Puffin,Safari,Seznam,YaBrowser,YE
fullVisitorId,,,,,,,,,,,,,,,,,,,,,
4823595352351,1,1.0,1.000000,1,0.0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5103959234087,10,8.0,0.800000,1,0.0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10278554503158,11,8.0,0.727273,1,0.0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20424342248747,17,13.0,0.764706,1,0.0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26722803385797,3,2.0,0.666667,1,0.0,0.0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [9]:
al.columns

Index(['hits', 'pageviews', 'page_hits', 'visitor_number', 'diff_days',
       'transac_1_0', 'channel.other', 'channel.affiliates', 'channel.direct',
       'channel.display',
       ...
       'Internet.Explorer', 'Maxthon', 'Nokia.1', 'Opera', 'browernew.Others',
       'Puffin', 'Safari', 'Seznam', 'YaBrowser', 'YE'],
      dtype='object', length=336)

### Splitting dataset

In [10]:
y=al.transac_1_0

In [11]:
data=al.drop(["transac_1_0"],axis=1)

In [12]:
data_train, data_test, y_train, y_test = train_test_split(data, y, test_size=0.3, random_state=123)

In [13]:
y_train.sum(), y_test.sum()

(6951.0, 3045.0)

## Classifications

### Random forest

__GridSearch__

In [14]:
rfc = RandomForestClassifier(random_state=1, min_samples_leaf=2, n_estimators=14)
m1 = rfc.fit(data_train, y_train)
p1 = m1.predict(data_test)

In [80]:
p1 = (m1.predict_proba(data_test)[:,1] >= 0.2).astype(bool)
confusion_matrix(p1, y_test)

array([[208190,    808],
       [  3016,   2237]])

In [79]:
imp1=pd.DataFrame(m1.feature_importances_, index=data_train.columns, columns=["rf_imp"])
imp1.sort_values(by="rf_imp", ascending=False)

,rf_imp
pageviews,0.207422
hits,0.173231
page_hits,0.084545
United.States,0.044038
Americas,0.036395
googleplex,0.030838
isMobile.0,0.030568
channel.referral,0.028539
diff_days,0.021639
Chrome,0.019020


### XGBoosting

In [16]:
xgb1 = XGBClassifier(max_depth=2, eta=1, silent=1, objective='binary:logistic',seed=2)
m3 = xgb1.fit(data_train, y_train)

In [17]:
p3 = (m3.predict_proba(data_test)[:,1] >= 0.2).astype(bool)
confusion_matrix(p3, y_test)

array([[208036,    732],
       [  3170,   2313]])

In [73]:
imp3=pd.DataFrame(m3.feature_importances_, index=data_train.columns, columns=["xgb_imp"])
imp3.sort_values(by="xgb_imp", ascending=False)

,xgb_imp
pageviews,0.558528
page_hits,0.083612
United.States,0.076923
Americas,0.053512
max.visitnumber,0.036789
isMobile.1,0.026756
hits,0.023411
google,0.020067
diff_days,0.020067
googleplex,0.016722


## Regression

In [84]:
pros=np.vstack([m1.predict_proba(data)[:,1],m3.predict_proba(data)[:,1]])

In [85]:
prob=pd.DataFrame(pros.T, index=data.index, columns=["rf_prob","xgb_prob"])

In [86]:
prob.iloc[:,0]

fullVisitorId
4823595352351           0.000000
5103959234087           0.000183
10278554503158          0.000000
20424342248747          0.001690
26722803385797          0.000000
27376579751715          0.003850
33471059618621          0.000000
35794135966385          0.000014
39460501403861          0.000000
40862739425590          0.004769
45417921646651          0.000000
47810105303746          0.000000
48421062322244          0.000128
52246474382344          0.000000
58970809397690          0.000000
61609983499251          0.000000
62267706107999          0.000000
62349695125717          0.000000
63834990385751          0.000000
64767209884626          0.000000
68403966359845          0.000000
72202462202136          0.000000
75746323734481          0.000000
77980744724824          0.000053
82001414223176          0.023244
84172700693338          0.000000
85059828173212          0.000239
90085033332104          0.000000
90134415725564          0.000000
93957001069502          0.000

In [87]:
data1=pd.concat([data,prob], axis=1, join='inner')

In [88]:
y1=all_new.log_revenue

In [89]:
data1_train, data1_test, y1_train, y1_test = train_test_split(data1, y1, random_state=2, test_size=0.3)

In [90]:
datao_train, datao_test, yo_train, yo_test = train_test_split(data, y1, random_state=2, test_size=0.3)

### Random Forest regressor

In [27]:
rfr = RandomForestRegressor()
rfr_m = rfr.fit(data1_train, y1_train)

In [28]:
rfr_p = rfr_m.predict(data1_test)

In [29]:
rfr_mae = MAE(rfr_p, y1_test)
rfr_mse = MSE(rfr_p, y1_test)
rfr_rmse = np.sqrt(rfr_mse)
rfr_mae, rfr_mse, rfr_rmse

(0.20052663092914386, 1.9681628538260472, 1.402912275884008)

In [292]:
pd.DataFrame(rfr_m.feature_importances_, index=data1_test.columns, columns=["imp"])

,imp
hits,0.031115
pageviews,0.021162
page_hits,0.036435
visitor_number,0.006698
diff_days,0.015923
channel.other,0.000013
channel.affiliates,0.000120
channel.direct,0.001758
channel.display,0.000640
channel.organicsearch,0.002854


### Linear regression

In [297]:
lr = LinearRegression()
lr_m = lr.fit(data1_train, y1_train)

In [298]:
lr_p = lr_m.predict(data1_test)

In [299]:
lr_mae = MAE(lr_p, y1_test)
lr_mse = MSE(lr_p, y1_test)
lr_rmse = np.sqrt(lr_mse)
lr_mae, lr_mse, lr_rmse

(42379.19954968211, 76437856896956.5, 8742874.635779498)